# <font color='orange' > <u> RestroRev Recommendation Engine</font> </u>

### <font color='orange'><u> We begin by importing the required libraries and the dataset</font></u>

In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import word_tokenize


In [2]:
data=pd.read_csv('zomato.csv')

In [3]:
data.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


### <font color='orange'><u> Preprocessing and cleanign the data set </font></u>

In [5]:
data['rate']=data['rate'].fillna(0.0)
data['rate']=data['rate'].replace(["NEW",'-'],'0.0')
data['rate']=data['rate'].apply( lambda x: str(x).split('/')[0])
data['rate'] = data.rate.astype(float)

In [7]:
data['rate'].unique()

array([4.1, 3.8, 3.7, 3.6, 4.6, 4. , 4.2, 3.9, 3.1, 3. , 3.2, 3.3, 2.8,
       4.4, 4.3, 0. , 2.9, 3.5, 2.6, 3.4, 4.5, 2.5, 2.7, 4.7, 2.4, 2.2,
       2.3, 4.8, 4.9, 2.1, 2. , 1.8])

### <font color='orange'><u> Feature selection for recommnedation </font></u>

In [8]:
data_new=data[['name','cuisines','listed_in(city)','rate']]
data_new['listed_in(city)'].value_counts(dropna = False).head(5)
#data_new['cuisines'] = data_new['cuisines'].dropna()
data_new = data_new.dropna(how='any', subset=['cuisines'])
data_sample = []

In [9]:
data_new.head()

,name,cuisines,listed_in(city),rate
0,Jalsa,"North Indian, Mughlai, Chinese",Banashankari,4.1
1,Spice Elephant,"Chinese, North Indian, Thai",Banashankari,4.1
2,San Churro Cafe,"Cafe, Mexican, Italian",Banashankari,3.8
3,Addhuri Udupi Bhojana,"South Indian, North Indian",Banashankari,3.7
4,Grand Village,"North Indian, Rajasthani",Banashankari,3.8


### <font color='orange'><u> Building the recommendation engine </font></u>

In [14]:
def restaurant_recommend_func(location, title):
    '''
    
    (location,title)---->Recommendations
    
    '''
    
    data_sample = data_new[data_new['listed_in(city)'] == location]
    # print(data_sample)
    # [till this point we have created/extracted the FEATURE data as per the user location]
#
    # index will be reset for cosine similarty index because Cosine similarty index has to be same value with result of tf-idf vectorize
    data_sample.reset_index(level=0, inplace=True, drop=True)
    #print(data_sample)
#
#     # Feature Extraction
    def process_cuisines(s):
        cuisines = s.split(',')   #split on commas
        words = [''.join(cuisine.split()) for cuisine in cuisines] # convert North Indian to NorthIndian
        return ' '.join(words)  #return Cusines seperated by single white space

    data_sample = data_sample.assign(Split=data_sample.apply(lambda row: process_cuisines(row[1]), axis=1))
    # TF-IDF vectorizer
    # Extracting Stopword
    tfidf = TfidfVectorizer(stop_words='english')
   
    tfidf_matrix = tfidf.fit_transform(data_sample['Split'])
    print(tfidf_matrix.shape)

    # Using for see Cosine Similarty scores
    #feature = tfidf.get_feature_names()
    #print(feature)
    # Cosine Similarity
    # Compute the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    print(cosine_sim.shape)
    # Column names are using for index
    #corpus_index = [n for n in data_sample['Split']]

    # Construct a reverse map of indices
    #indices = pd.Series(data_sample.index, index=data_sample['name']).drop_duplicates()

    # index of the restaurant matchs the cuisines
    #idx = indices[title]
    #print(idx)
    # rate added with cosine score in sim_score list.
    sim_scores = []
    
    self_index = data_sample[data_sample['name'] == title].index
    #print(cosine_sim[self_index].mean(axis=0))
    
    #print(cosine_sim[0])
    for i, j in enumerate(cosine_sim[self_index].mean(axis=0)):
        k = data_sample['rate'].iloc[i]
        sim_scores.append((i, j, k))

    # Sort the names based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: (x[1], x[2]), reverse=True)
    # 10 similar cuisines
    sim_scores = sim_scores[0:10]
    rest_indices = [i[0] for i in sim_scores]

    data_x = data_sample.iloc[rest_indices]
    
    data_x = data_x.assign(cosine_score=1)
    
    
    return (data_x.drop_duplicates())

### <font color='orange'> <u> Output Generation </font></u>

In [15]:
y = restaurant_recommend_func('Banashankari','Jalsa')
y

(860, 51)
(860, 860)


,name,cuisines,listed_in(city),rate,Split,cosine_score
0,Jalsa,"North Indian, Mughlai, Chinese",Banashankari,4.1,NorthIndian Mughlai Chinese,1
814,Pramukh Family Restaurant,"North Indian, Chinese, Mughlai",Banashankari,3.9,NorthIndian Chinese Mughlai,1
159,Wahab,"North Indian, Mughlai, Chinese",Banashankari,3.7,NorthIndian Mughlai Chinese,1
609,Food Box Cafe,"Mughlai, North Indian, Chinese",Banashankari,3.6,Mughlai NorthIndian Chinese,1
432,7 Hills Garden Restaurant,"North Indian, Mughlai, Chinese",Banashankari,0.0,NorthIndian Mughlai Chinese,1
38,Empire Restaurant,"North Indian, Mughlai, South Indian, Chinese",Banashankari,4.4,NorthIndian Mughlai SouthIndian Chinese,1
812,Shrinidhi Military Hotel,"North Indian, Mughlai",Banashankari,3.3,NorthIndian Mughlai,1


In [17]:
restaurant_recommend_func('Basavanagudi','Grand Village')

(1266, 61)
(1266, 1266)


,name,cuisines,listed_in(city),rate,Split,cosine_score
925,Kesar Rajasthani Parotha Point,"North Indian, Rajasthani",Basavanagudi,3.9,NorthIndian Rajasthani,1
2,Grand Village,"North Indian, Rajasthani",Basavanagudi,3.8,NorthIndian Rajasthani,1
253,Sri Rajasthani Foods,"Rajasthani, North Indian",Basavanagudi,3.5,Rajasthani NorthIndian,1
176,Bikaner Jn,Rajasthani,Basavanagudi,3.9,Rajasthani,1
460,Desi Rasoi,"Biryani, North Indian, Chinese, Rajasthani",Basavanagudi,3.8,Biryani NorthIndian Chinese Rajasthani,1
147,Mini Punjabi Dhaba,North Indian,Basavanagudi,4.2,NorthIndian,1
170,Punjabi by Nature 2.0,North Indian,Basavanagudi,4.2,NorthIndian,1
467,Kapoor's Cafe,North Indian,Basavanagudi,4.2,NorthIndian,1
469,Chulha Chauki Da Dhaba,North Indian,Basavanagudi,4.2,NorthIndian,1
